# Task1

#
Query of selecting candidate key from master and teams:
SELECT
    playerID,
    COUNT(*) as count
FROM
    lahman2016.master
GROUP BY playerID
ORDER BY count DESC;

SELECT
    COUNT(*) as count
FROM
    lahman2016.teams
GROUP BY yearID,teamID
ORDER BY count DESC;
with the count is no more than 1, we can use these columns together as candidate key and primary key.
#
Add primary keys using mysql workbench using alter table
ALTER TABLE `lahman2016`.`master` 
CHANGE COLUMN `playerID` `playerID` VARCHAR(255) NOT NULL ,
ADD PRIMARY KEY (`playerID`);

ALTER TABLE `lahman2016`.`teams` 
CHANGE COLUMN `yearID` `yearID` INT(11) NOT NULL ,
CHANGE COLUMN `teamID` `teamID` VARCHAR(255) NOT NULL ,
ADD PRIMARY KEY (`yearID`, `teamID`);

#
In master, I choose playerID as primary key
In teams, I choose yearID,teamID as primary key
In appearances, I choose playerID,teamID,yearID as primary key
In batting, I choose yearID,playerID,stint,teamID as primary key
In pitching, I choose yearID,playerID,stint,teamID as primary key
In fielding, I choose yearID,playerID,stint,POS,teamID as primary key
In salaries, I choose yearID,playerID,teamID as primary key

#
Create foriegn keys
From master choose playerID as reference of all tables that has playerID
ALTER TABLE `lahman2016`.`appearances` 
ADD INDEX `playerID_idx` (`playerID` ASC);
ALTER TABLE `lahman2016`.`appearances` 
ADD CONSTRAINT `playerID`
  FOREIGN KEY (`playerID`)
  REFERENCES `lahman2016`.`master` (`playerID`)
  ON DELETE NO ACTION
  ON UPDATE NO ACTION;
  
From teams choose teamID and yearID as reference of all tables that has teamID and yearID
ALTER TABLE `lahman2016`.`appearances` 
ADD INDEX `team and year_idx` (`yearID` ASC, `teamID` ASC);
ALTER TABLE `lahman2016`.`appearances` 
ADD CONSTRAINT `team and year`
  FOREIGN KEY (`yearID` , `teamID`)
  REFERENCES `lahman2016`.`teams` (`yearID` , `teamID`)
  ON DELETE NO ACTION
  ON UPDATE NO ACTION;

# Task2

In [8]:
#connect to mysql
import pymysql.cursors
import json


def connect():
    connection = pymysql.connect(host='localhost',
                                 user='yuekun',
                                 password='777862',
                                 db='lahman2016',
                                 charset='utf8mb4',
                                 cursorclass=pymysql.cursors.DictCursor)
    return connection

def disconnect(c):
    c.close()

def pretty_print(r):
    r = json.dumps(r, indent=4, sort_keys=True)
    print("Result = ", r)
    return r

In [3]:
#This is function to run query
def run_sql(sql):
    connection = connect()

    with connection.cursor() as cursor:
        # Execute the SQL statement.
        print("Connected.")
        cursor.execute(sql)
        
        # We know that this is a primary key. We only need to fetch ONE row.
        result = cursor.fetchall()
        return result

In [108]:
#this is the function used to judge whether the input path is legal
def test_path(sql,query,resource,relation):
    if len(sql) == 0:
        sql1 = "SELECT *" +" FROM " + resource + " LIMIT 1"
    elif query != None:
        sql1 = "SELECT *" +" FROM " + resource +" WHERE "+ sql + " LIMIT 1"
    else:
        sql1 = "SELECT *" +" FROM " + resource + sql + " LIMIT 1"
    try:
        run_sql(sql1)
        if relation != "":
            if len(sql) == 0:
                sql2 = "SELECT *" +" FROM " + relation + " LIMIT 1"
            else:
                sql2 = "SELECT *" +" FROM " + relation +" WHERE "+ sql + " LIMIT 1"
            run_sql(sql2)
            if query != None:
                query = query.split(' ')
                if query[len(query)-2] == 'AND':
                    del query[len(query)-2]
                query = ' '.join(query)
                sql3 = "SELECT *" +" FROM " + relation + query + " LIMIT 1"
                run_sql(sql3)
        else:
            if query != None:
                query = query.split(' ')
                if query[len(query)-2] == 'AND':
                    del query[len(query)-2]
                query = ' '.join(query)
                sql3 = "SELECT *" +" FROM " + resource + query + " LIMIT 1"
                run_sql(sql3)
        return True
    except:
        print('{ "error" : "Invalid path"}')
        return False

In [28]:
#This is function used to do field judgement
#Caps sensitive
def judge_field(resource,fields):
    if fields == '*':
        return True
    sql = "SHOW COLUMNS FROM " + resource
    result = run_sql(sql)
    f = set()
    for r in result:
        f.add(r["Field"])
    fields = fields.split(',')
    error = ""
    for field in fields:
        if field not in f:
            error += field + ", "
    if len(error) != 0:
        print('{ "error" : "Field not found", "field" : "' + error + '"}' )
        return False
    else:
        return True

In [110]:
#main function
def data_access_service(path,query,fields):
    path = path.split('/')
    resource = path[1]
    sql = ""
    relation = ""
    #transfer input to query language
    if query != None:
        query = query.replace(" ", "")
        query = query.replace("AND"," AND ")
        query = " WHERE " + query + " AND "
    try:
        collections = path[2]
        collection = collections.split('-')
        if query == None:
            sql += " WHERE "
        sql += "playerID='" + collection[0] + "' "
        if len(collection) > 1:
            sql += "AND " + "yearID='" + collection[1] + "' "
        if len(collection) > 2:
            sql += "AND "+ "stint='" + collection[2] + "' "
        try:
            relation = path[3]
        except:
            pass
    except:
        pass
    if not test_path(sql,query,resource,relation):
        return ('{}')
    if query != None:
        sql = query + sql
    if len(path) > 3:
        resource = path[3]
    if not judge_field(resource,fields):
        return ('{}')
    sql = "SELECT "+ fields +" FROM " + resource + sql
    sql = sql.split(' ')
    if sql[len(sql)-2] == 'AND':
        del sql[len(sql)-2]
    sql = ' '.join(sql)
    print("query is:" + sql)
    result = run_sql(sql)
    try:
        result = run_sql(sql)
        result = pretty_print(result)
        result = result.replace("\n","")
        return ('{' + result + '}')
    except:
        #If there is no error above, print internal error
        print('{ "error" : "Internal error" }')
        return ('{}')

In [126]:
#test case 1
data_access_service("/batting/willite01-1940-1", None, "G,AB,H")

Connected.
Connected.
query is:SELECT G,AB,H FROM batting WHERE playerID='willite01' AND yearID='1940' AND stint='1' 
Connected.
Connected.
Result =  [
    {
        "AB": 561,
        "G": 144,
        "H": 193
    }
]


'{[    {        "AB": 561,        "G": 144,        "H": 193    }]}'

In [111]:
#test case 2
data_access_service('/batting', "playerID = 'willite01' AND yearID > 1946", "*")

Connected.
Connected.
query is:SELECT * FROM batting WHERE playerID='willite01' AND yearID>1946 
Connected.
Connected.
Result =  [
    {
        "2B": 40,
        "3B": 9,
        "AB": 528,
        "BB": 162,
        "CS": 1,
        "G": 156,
        "GIDP": "10",
        "H": 181,
        "HBP": "2",
        "HR": 32,
        "IBB": "",
        "R": 125,
        "RBI": 114,
        "SB": 0,
        "SF": "",
        "SH": "1",
        "SO": 47,
        "lgID": "AL",
        "playerID": "willite01",
        "stint": 1,
        "teamID": "BOS",
        "yearID": 1947
    },
    {
        "2B": 44,
        "3B": 3,
        "AB": 509,
        "BB": 126,
        "CS": 0,
        "G": 137,
        "GIDP": "10",
        "H": 188,
        "HBP": "3",
        "HR": 25,
        "IBB": "",
        "R": 124,
        "RBI": 127,
        "SB": 4,
        "SF": "",
        "SH": "0",
        "SO": 41,
        "lgID": "AL",
        "playerID": "willite01",
        "stint": 1,
        "teamID": "BOS

'{[    {        "2B": 40,        "3B": 9,        "AB": 528,        "BB": 162,        "CS": 1,        "G": 156,        "GIDP": "10",        "H": 181,        "HBP": "2",        "HR": 32,        "IBB": "",        "R": 125,        "RBI": 114,        "SB": 0,        "SF": "",        "SH": "1",        "SO": 47,        "lgID": "AL",        "playerID": "willite01",        "stint": 1,        "teamID": "BOS",        "yearID": 1947    },    {        "2B": 44,        "3B": 3,        "AB": 509,        "BB": 126,        "CS": 0,        "G": 137,        "GIDP": "10",        "H": 188,        "HBP": "3",        "HR": 25,        "IBB": "",        "R": 124,        "RBI": 127,        "SB": 4,        "SF": "",        "SH": "0",        "SO": 41,        "lgID": "AL",        "playerID": "willite01",        "stint": 1,        "teamID": "BOS",        "yearID": 1948    },    {        "2B": 39,        "3B": 3,        "AB": 566,        "BB": 162,        "CS": 1,        "G": 155,        "GIDP": "22",        "H": 19

In [113]:
#test case 3
data_access_service("/master/willite01/batting", "yearID > 1946", "*")

Connected.
Connected.
Connected.
query is:SELECT * FROM batting WHERE yearID>1946 AND playerID='willite01' 
Connected.
Connected.
Result =  [
    {
        "2B": 40,
        "3B": 9,
        "AB": 528,
        "BB": 162,
        "CS": 1,
        "G": 156,
        "GIDP": "10",
        "H": 181,
        "HBP": "2",
        "HR": 32,
        "IBB": "",
        "R": 125,
        "RBI": 114,
        "SB": 0,
        "SF": "",
        "SH": "1",
        "SO": 47,
        "lgID": "AL",
        "playerID": "willite01",
        "stint": 1,
        "teamID": "BOS",
        "yearID": 1947
    },
    {
        "2B": 44,
        "3B": 3,
        "AB": 509,
        "BB": 126,
        "CS": 0,
        "G": 137,
        "GIDP": "10",
        "H": 188,
        "HBP": "3",
        "HR": 25,
        "IBB": "",
        "R": 124,
        "RBI": 127,
        "SB": 4,
        "SF": "",
        "SH": "0",
        "SO": 41,
        "lgID": "AL",
        "playerID": "willite01",
        "stint": 1,
        "te

'{[    {        "2B": 40,        "3B": 9,        "AB": 528,        "BB": 162,        "CS": 1,        "G": 156,        "GIDP": "10",        "H": 181,        "HBP": "2",        "HR": 32,        "IBB": "",        "R": 125,        "RBI": 114,        "SB": 0,        "SF": "",        "SH": "1",        "SO": 47,        "lgID": "AL",        "playerID": "willite01",        "stint": 1,        "teamID": "BOS",        "yearID": 1947    },    {        "2B": 44,        "3B": 3,        "AB": 509,        "BB": 126,        "CS": 0,        "G": 137,        "GIDP": "10",        "H": 188,        "HBP": "3",        "HR": 25,        "IBB": "",        "R": 124,        "RBI": 127,        "SB": 4,        "SF": "",        "SH": "0",        "SO": 41,        "lgID": "AL",        "playerID": "willite01",        "stint": 1,        "teamID": "BOS",        "yearID": 1948    },    {        "2B": 39,        "3B": 3,        "AB": 566,        "BB": 162,        "CS": 1,        "G": 155,        "GIDP": "22",        "H": 19

In [114]:
#test case wrong field
data_access_service("/batting/willite01-1940-1", None, "fake-field")

Connected.
Connected.
{ "error" : "Field not found", "field" : "fake-field, "}


'{}'

In [118]:
#test case wrong path1
data_access_service("/fake_path","yearID > 1946", "*")

Connected.
{ "error" : "Invalid path"}


'{}'

In [124]:
#test case wrong path2
data_access_service("/master/willite01", "yearID > 1946", "*")

Connected.
Connected.
{ "error" : "Invalid path"}


'{}'

In [125]:
#test case wrong path3
data_access_service("/batting/willite01-1940-1", "fake", "G,AB,H")

Connected.
Connected.
{ "error" : "Invalid path"}


'{}'